In [1]:
import dlib
import cv2
import numpy as np
import argparse
import imutils
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils

In [2]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1],eye[5])
    B = dist.euclidean(eye[2],eye[4])
    
    C = dist.euclidean(eye[0],eye[3])
    aspect_ratio = (A+B)/(2.0*C)
    return aspect_ratio

In [3]:
# shape_predictor_68_face_landmarks.dat

In [4]:
threshold = 0.25
frames = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [5]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

In [6]:
cap = cv2.VideoCapture(0)
flag = 0
while True:
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray,subject)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        left_aspect_ratio = eye_aspect_ratio(leftEye)
        right_aspect_ratio = eye_aspect_ratio(rightEye)
        aspect_ratio = (left_aspect_ratio + right_aspect_ratio)/2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        if aspect_ratio < threshold:
            flag+=1
            if flag>=frames:
                cv2.put_text(frame,'Alert!!!!',(15,25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1)
            else:
                flag = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        cap.release()
        break

In [8]:
def drowsiness_detector(file):
    image = cv2.imread(file)
    image = cv2.resize(image,(500,500))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray,subject)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        left_aspect_ratio = eye_aspect_ratio(leftEye)
        right_aspect_ratio = eye_aspect_ratio(rightEye)
        aspect_ratio = (left_aspect_ratio + right_aspect_ratio)/2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(image, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(image, [rightEyeHull], -1, (0, 255, 0), 1)
        if aspect_ratio < threshold:
            cv2.putText(image,'Alert!!!!',(15,25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1)
            cv2.putText(image,str(aspect_ratio),(15,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1)
        else:
            cv2.putText(image,'All Okay!!!',(15,25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
            cv2.putText(image,str(aspect_ratio),(15,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1)
    cv2.imshow("Frame", image)
    cv2.waitKey(0)

In [9]:
drowsiness_detector('sleepy_eyes.jfif')

In [10]:
# non-sleepy.jfif
drowsiness_detector('non-sleepy.jfif')